# **Resolución TP1: Algoritmos de búsqueda en Torre de Hanoi**

## 1. ¿Cuáles son los PEAS de este problema?

- **Perfomance**: partiendo del estado inicial (cinco discos apilados de en varilla 1) moverlos a varilla 3, respetando las reglas del juego.

- **Enviroment**: cinco discos de distintos tamaños, base con tres varillas.

- **Actuators**:

- **Sensors**: detección del estado actual de las varillas (discos en cada una y orden de esos discos).

## 2. ¿Cuáles son las propiedades del entorno de trabajo?

- **Totalmente observable**: el agente puede ver en todo momento el estado del entorno (posición de los discos en cada varilla).

- **Determinista**: el siguiente estado esta determinado por el estado actual y la acción que ejecuta el agente.

- **Secuencial**: la decisión actual puede afectar las decisiones futuras.

- **Estático**: el entorno no se modifica mientras el agente delibera.

- **Discreto**: tiene un número finito de estados distintos.

- **Agente individual**

## 3. En el contexto de este problema, defina los siguientes conceptos:
- **Estado**: Una configuración específica de los discos en las tres varillas, indicando en qué varilla está cada disco y su orden.
- **Espacio de estados**: El conjunto de todos los posibles estados alcanzables mediante movimientos válidos de los discos en las distintas varillas.
- **Árbol de búsqueda**: Representación de todos los estados posibles (nodos) y cómo llegamos a ellos (rama). Se recorren los caminos desde el estado inicias hasta llegar a al estado objetivo.
- **Nodo de búsqueda**: Configuración intermedia de los discos entre el estado inicial y el final representada por un nodo en el árbol de búsqueda. En este nodo se sabe cómo se llegó a este estado, es decir, cuáles fueron las configuraciones y acciones previas (camino de la rama), qué número de movimiento es el actual (profundidad) y cuál es el costo acumulado.
- **Objetivo**: Estado donde todos los discos están aplicados en la tercera varilla en forma de pirámide.
- **Acción**: Movimiento permitido (rama del árbol) de uno de los discos hacia alguna de las varillas que no es la actual mientras se siga con las reglas del juego.
- **Frontera**: Conjunto de nodos generados pero aún no explorados. Representa los posibles caminos a seguir en la búsqueda de la solución. Según el algoritmo que se use, puede ser una pila o una cola de estados a explorar.

## 4.Implemente algún método de búsqueda.

En este caso se eligió implementar el método de costo uniforme. Para asignar el costo de cada camino se aplicó la fórmula:

_<center>costo_de_camino = 2^(numero_de_disco - 1)</center>_

donde el número del disco va, en este problema en particular, del 1 al 5, 1 para el más pequeño y 5 para el más grande, por lo que mover un disco más grande es más costoso.

### Implementación


In [2]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from queue import PriorityQueue

def calculate_cost(disk_number):
    move_cost = 2 ** (disk_number - 1)
    return move_cost

def uniform_cost_search(disks_amount):
    initial_disks = list(range(disks_amount, 0, -1))
    initial_state = StatesHanoi(initial_disks, [], [], max_disks=disks_amount)
    goal_state = StatesHanoi([], [], initial_disks, max_disks=disks_amount)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)
    #Para prioridad
    priority_queue = PriorityQueue()
    #Para guardar nodos visitados
    visited_nodes = {}
    # Raíz del árbol
    root = NodeHanoi(state=initial_state)
    priority_queue.put((0, root))

    while not priority_queue.empty():
        priority_value, new_node = priority_queue.get()
        frontier_nodes=new_node.expand(problem=problem)
        if problem.goal_test(new_node.state):
            last_node = new_node
            metrics = {
                "last_node": last_node,
                "max_depth": new_node.depth,
                "cost_total": new_node.state.accumulated_cost,
            }
            solution= last_node.solution()
            return last_node, solution , metrics

        if new_node.state in visited_nodes and visited_nodes[new_node.state] <= priority_value:
            continue

        # Marcar el nodo como visitado con su costo acumulado
        visited_nodes[new_node.state] = priority_value

        for node in frontier_nodes:
            total_cost=priority_value + calculate_cost(node.action.disk)
            priority_queue.put((total_cost, node))
        
    
    print("No se encontró la solución")
    metrics = {
                "last_node": None,
                "max_depth": None,
                "cost_total": None,
            }
    return None, None , metrics


### Solución y métricas


In [ ]:
last_node, solution, metrics= uniform_cost_search(5)

for act in solution:
    print(act)

print(metrics)

## 5. ¿Cuál es la complejidad teórica en tiempo y memoria del algoritmo elegido?
- Complejidad teórica en tiempo: En el peor caso, la complejidad es O(b^d), donde b es el factor de ramificación (cantidad máxima de movimientos posibles desde un estado), d es la profundidad del nodo objetivo (cantidad de movimientos para llegar a la solución). Esto se debe a que el algoritmo puede llegar a expandir todos los nodos del espacio de estados antes de encontrar la solución óptima.
- Complejidad teórica en memoria: También es O(b^d), ya que debe almacenar todos los nodos generados en la frontera (cola de prioridad) y los visitados.
- Aplicado a nuestro problema el espacio de estados es O(3^n), ya que cada disco puede estar en cualquiera de las 3 varillas. Por lo tanto, tanto el tiempo como la memoria crecen exponencialmente con la cantidad de discos.

## 6. A nivel de implementación, ¿cuánto tiempo y memoria utiliza el algoritmo? (Se recomienda ejecutarlo 10 veces y calcular el promedio y el desvío estándar de ambas métricas).

#### Código para medición de tiempo de ejecución y uso de memoria


In [ ]:
%%timeit

## Tiempo de ejecución
last_node, solution, metrics= uniform_cost_search(5)

In [ ]:
import tracemalloc

tracemalloc.start()

metrics = uniform_cost_search(5)

_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Luego de ejecutar el código 10 veces y realizar el cálculo del promedio y desvío estándar para cada métrica, se obtuvieron los siguientes resultados:

**Tiempo de ejecución**

_Promedio:_ 640,8 ms  
_Desvío estándar:_ 61.86 ms

**Memoria utilizada**

_Promedio:_ 0.584 MB  
_Desvío estándar:_ 0.157 MB


Código para generar json para simulador


In [6]:
last_node, solution, metrics= uniform_cost_search(5)
last_node.generate_solution_for_simulator()